Piotr Ostaszewski (325697)

# Wskazanie optymalnej lokalizacji farmy fotowoltaicznej

In [1]:
#1.  ************ !!!!!!!  Paths  !!!!!!! ************
workspace_path = r"\Project1\workspace.gdb"

gmina_name = "Świeradów-Zdrój"; gmina_name_field = "JPT_NAZWA_"
gminy_prg_path = r"\Project1\dane\02_GraniceAdministracyjne\A03_Granice_gmin.shp"

nmt_folder_path = r"\Project1\dane\nmt"
bdot_folders_paths_list = [r"\Project1\dane\0210_SHP", r"\Project1\dane\0212_SHP"]

wezly_distance_raster_path = r"\Project1\dane\qWezly\qWezly.tif"
dzialki_path = r"\Project1\dane\qDzialki\dzialki_2180_gmina_Clip.shp"
#    ***************************************************

In [2]:
#2. WORKSPACE INITIAL CONFIGURATION
import arcpy
import numpy as np
from pathlib import Path

arcpy.env.workspace = workspace_path
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRF2000-PL CS92")
arcpy.env.cellSize = 5
arcpy.env.overwriteOutput = True


In [3]:
#3. PROCESSING EXTENT - END OF WORKSPACE CONFIGURATION
# paths
gmina = gmina_name; gmina_field = gmina_name_field
gminy_file = gminy_prg_path

# getting the gmina shape
with arcpy.da.SearchCursor(gminy_file, [gmina_field, "SHAPE@"]) as cursor:
    for row in cursor:
        random_gmina_from_file = row[0]
        if random_gmina_from_file == gmina:
            arcpy.CopyFeatures_management(row[1], "gmina_sam")

# buffer
arcpy.analysis.Buffer("gmina_sam", "gminaBuffer", "200 Meters")

# end of workspace configuration
arcpy.env.extent = "gminaBuffer"
arcpy.env.mask = "gminaBuffer"

In [2]:
#4. fast configuration, but there's also need for cell to be exeuted before:
# ****************** !!!!!!!! Paths !!!!!!!! ***************
#                       !                 !
#                       !                 !
#                       !                 !
#                       !                 !
import arcpy
import numpy as np
from pathlib import Path

arcpy.env.workspace = workspace_path
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRF2000-PL CS92")
arcpy.env.cellSize = 5
arcpy.env.overwriteOutput = True
arcpy.env.extent = "gminaBuffer"
arcpy.env.mask = "gminaBuffer"

In [5]:
#5. NMT
# path
nmt_folder = nmt_folder_path
nmt_file_list = [str(file) for file in Path(nmt_folder).rglob('*')]
asc_list = []
for filename in nmt_file_list:
    if filename.endswith(".asc"):
        asc_list.append(arcpy.Raster(filename))

nmt = arcpy.ia.Merge(asc_list, "MEAN")
nmt = arcpy.sa.ExtractByMask(nmt, "gminaBuffer")

nmt.save("nmt")


In [ ]:
#6. BDOT10k
import os

# ścieżki
bdot_folders = bdot_folders_paths_list
l_swrs = []; l_ptwp = []; l_bubd = []; l_ptlz = []; l_skjz = []; l_skdr = []
# reszta do pokrycia
l_ptzb = []; l_ptrk = []; l_ptut = []; l_pttr = []; l_ptkm = []; l_ptgn = []; l_ptpl = []; l_ptso = []; l_ptwz = []; l_ptnz = []
l_sktr = []

# prąd
l_suln = []

# get shapefiles
for folder in bdot_folders:
    # rename bad filenames
    bdot_file_list = [str(file) for file in Path(folder).rglob('*')]
    for filename_bad in bdot_file_list:
        filename = ""
        for i, ch in enumerate(filename_bad):
            if ch == "." and i < len(filename_bad) - 5:
                filename += '_'
            else:
                filename += ch
    
        os.rename(filename_bad, filename)
        
        
        # filename = filename.replace("\\", "\\\\")
        if filename.endswith(".shp"):
            if "OT_SWRS_L" in filename:
                l_swrs.append(filename)
            elif "OT_PTWP_A" in filename:
                l_ptwp.append(filename)
            elif "OT_BUBD_A" in filename:
                l_bubd.append(filename)
            elif "OT_PTLZ_A" in filename:
                l_ptlz.append(filename)
            elif "OT_SKJZ_L" in filename:
                l_skjz.append(filename)
            elif "OT_SKDR_L" in filename:
                l_skdr.append(filename)
            # reszta do pokrycia
            elif "OT_PTZB_A" in filename:
                l_ptzb.append(filename)
            elif "OT_PTRK_A" in filename:
                l_ptrk.append(filename)
            elif "OT_PTUT_A" in filename:
                l_ptut.append(filename)
            elif "OT_PTTR_A" in filename:
                l_pttr.append(filename)
            elif "OT_PTKM_A" in filename:
                l_ptkm.append(filename)
            elif "OT_PTGN_A" in filename:
                l_ptgn.append(filename)
            elif "OT_PTPL_A" in filename:
                l_ptpl.append(filename)
            elif "OT_PTSO_A" in filename:
                l_ptso.append(filename)
            elif "OT_PTWZ_A" in filename:
                l_ptwz.append(filename)
            elif "OT_PTNZ_A" in filename:
                l_ptnz.append(filename)
            # tory dla terenów komunikacyjnych
            elif "OT_SKTR_L" in filename:
                l_sktr.append(filename)
            # prąd
            elif "OT_SULN_L" in filename:
                l_suln.append(filename)


# merge layers
arcpy.management.Merge(l_swrs, "uncroppedSWRS_L")
arcpy.management.Merge(l_ptwp, "uncroppedPTWP_A")
arcpy.management.Merge(l_bubd, "uncroppedBUBD_A")
arcpy.management.Merge(l_ptlz, "uncroppedPTLZ_A")
arcpy.management.Merge(l_skjz, "uncroppedSKJZ_L")
arcpy.management.Merge(l_skdr, "uncroppedSKDR_L")

# reszta do pokrycia
arcpy.management.Merge(l_ptzb, "uncroppedPTZB_A")
arcpy.management.Merge(l_ptrk, "uncroppedPTRK_A")
arcpy.management.Merge(l_ptut, "uncroppedPTUT_A")
arcpy.management.Merge(l_pttr, "uncroppedPTTR_A")
arcpy.management.Merge(l_ptkm, "uncroppedPTKM_A")
arcpy.management.Merge(l_ptgn, "uncroppedPTGN_A")
arcpy.management.Merge(l_ptpl, "uncroppedPTPL_A")
arcpy.management.Merge(l_ptso, "uncroppedPTSO_A")
arcpy.management.Merge(l_ptwz, "uncroppedPTWZ_A")
arcpy.management.Merge(l_ptnz, "uncroppedPTNZ_A")

arcpy.management.Merge(l_sktr, "uncroppedSKTR_L")
arcpy.management.Merge(l_suln, "uncroppedSULN_L")

# clip layers
arcpy.analysis.Clip("uncroppedSWRS_L", "gminaBuffer", "SWRS_L")
arcpy.analysis.Clip("uncroppedPTWP_A", "gminaBuffer", "PTWP_A")
arcpy.analysis.Clip("uncroppedBUBD_A", "gminaBuffer", "BUBD_A")
arcpy.analysis.Clip("uncroppedPTLZ_A", "gminaBuffer", "PTLZ_A")
arcpy.analysis.Clip("uncroppedSKJZ_L", "gminaBuffer", "SKJZ_L")
arcpy.analysis.Clip("uncroppedSKDR_L", "gminaBuffer", "SKDR_L")

# reszta do pokrycia
arcpy.analysis.Clip("uncroppedPTZB_A", "gminaBuffer", "PTZB_A")
arcpy.analysis.Clip("uncroppedPTRK_A", "gminaBuffer", "PTRK_A")
arcpy.analysis.Clip("uncroppedPTUT_A", "gminaBuffer", "PTUT_A")
arcpy.analysis.Clip("uncroppedPTTR_A", "gminaBuffer", "PTTR_A")
arcpy.analysis.Clip("uncroppedPTKM_A", "gminaBuffer", "PTKM_A")
arcpy.analysis.Clip("uncroppedPTGN_A", "gminaBuffer", "PTGN_A")
arcpy.analysis.Clip("uncroppedPTPL_A", "gminaBuffer", "PTPL_A")
arcpy.analysis.Clip("uncroppedPTSO_A", "gminaBuffer", "PTSO_A")
arcpy.analysis.Clip("uncroppedPTWZ_A", "gminaBuffer", "PTWZ_A")
arcpy.analysis.Clip("uncroppedPTNZ_A", "gminaBuffer", "PTNZ_A")

arcpy.analysis.Clip("uncroppedSKTR_L", "gminaBuffer", "SKTR_L")
arcpy.analysis.Clip("uncroppedSULN_L", "gminaBuffer", "SULN_L")

In [7]:
#7. I odległość od rzek i zbiorników wodnych
arcpy.analysis.Buffer("SWRS_L", "SWRS_LBuffer", "2 Meters", "FULL", "ROUND", "ALL")
arcpy.management.Merge(["SWRS_LBuffer", "PTWP_A"], "wody")

wodyDistance = arcpy.sa.EucDistance("wody")
wodyDistance.save("wodyDistance")

wodyDistance_sam = arcpy.sa.ExtractByMask(wodyDistance, "gmina_sam")
wodyDistance_sam.save("wodyDistance_sam")

wodyFuzzy_far = arcpy.sa.FuzzyMembership("wodyDistance_sam", arcpy.sa.FuzzyLinear( 500, 100))
wodyFuzzy_far.save("wodyFuzzy_far")

wodyFuzzy_near = arcpy.sa.FuzzyMembership("wodyDistance_sam", arcpy.sa.FuzzyLinear( 100, 100.01))
wodyFuzzy_near.save("wodyFuzzy_near")

o_wodyFuzzy = arcpy.ia.Merge(["wodyFuzzy_far", "wodyFuzzy_near"], "MIN")
o_wodyFuzzy.save("o_wodyFuzzy")

wodySharp = arcpy.sa.Reclassify(
    in_raster="wodyDistance_sam",
    reclass_field="VALUE",
    remap="0 100 0;100 1000000 1",
)
wodySharp.save("o_wodySharp")

In [8]:
#8. II odległość od budynków mieszkalnych
budynkiM = arcpy.management.SelectLayerByAttribute('BUBD_A', "NEW_SELECTION",
                                                     "FOBUD = 'budynki mieszkalne'")
arcpy.management.CopyFeatures(budynkiM, 'budynkiM')

budynkiDistance = arcpy.sa.EucDistance('budynkiM')
budynkiDistance.save('budynkiDistance')

budynkiDistance_sam = arcpy.sa.ExtractByMask(budynkiDistance, 'gmina_sam')
budynkiDistance_sam.save('budynkiDistance_sam')

o_budynkiFuzzy = arcpy.sa.FuzzyMembership('budynkiDistance_sam', arcpy.sa.FuzzyLinear( 150, 500))
o_budynkiFuzzy.save('o_budynkiFuzzy')

budSharp = arcpy.sa.Reclassify(
    in_raster="budynkiDistance_sam",
    reclass_field="VALUE",
    remap="0 150 0;150 1000000 1",
)
budSharp.save("o_budynkiSharp")

In [9]:
#9. III pokrycie terenu
lasyDistance = arcpy.sa.EucDistance('PTLZ_A')
lasyDistance.save('lasyDistance')

lasyDistance_sam = arcpy.sa.ExtractByMask(lasyDistance, 'gmina_sam')
lasyDistance_sam.save('lasyDistance_sam')

o_lasyFuzzy = arcpy.sa.FuzzyMembership('lasyDistance_sam', arcpy.sa.FuzzyLinear( 15, 100))
o_lasyFuzzy.save('o_lasyFuzzy')

lasySharp = arcpy.sa.Reclassify(
    in_raster="lasyDistance_sam",
    reclass_field="VALUE",
    remap="0 15 0;15 1000000 1",
)
lasySharp.save("o_lasySharp")

In [10]:
#10. IV dostęp do dróg utwardzonych
expression = "get_weight(!MATE_NAWIE!)"
codeblock = """
def get_weight(naw):
    if naw == 'beton':
        return 2
    elif naw == 'bruk':
        return 1
    elif naw == 'grunt naturalny':
        return 0
    elif naw == 'kostka kamienna':
        return 1
    elif naw == 'kostka prefabrykowana':
        return 2
    elif naw == 'masa bitumiczna':
        return 2
    elif naw == 'płyty betonowe':
        return 2
    elif naw == 'żwir':
        return 1
    elif naw == 'tłuczeń':
        return 1
    elif naw == 'inny':
        return 0
"""

arcpy.management.AddField('SKDR_L', 'waga_nawie', "SHORT")
arcpy.management.CalculateField('SKDR_L', 'waga_nawie', expression, "PYTHON3", codeblock)

drogiLineDensity = arcpy.sa.LineDensity('SKDR_L', 'waga_nawie', search_radius=200, area_unit_scale_factor='SQUARE_KILOMETERS')
drogiLineDensity.save("drogiLineDensity")

drogiLineDensity_sam = arcpy.sa.ExtractByMask(drogiLineDensity, 'gmina_sam')
drogiLineDensity_sam.save('drogiLineDensity_sam')

o_drogiFuzzy = arcpy.sa.FuzzyMembership('drogiLineDensity_sam', arcpy.sa.FuzzyLinear( 0, 15))
o_drogiFuzzy.save('o_drogiFuzzy')

In [11]:
#11. V V nachylenie stoków
slope = arcpy.sa.Slope("nmt", "DEGREE")
slope.save("slope")

slope_sam = arcpy.sa.ExtractByMask(slope, 'gmina_sam')
slope_sam.save('slope_sam')

o_slopeFuzzy = arcpy.sa.FuzzyMembership('slope_sam', arcpy.sa.FuzzyLinear( 15, 5))
o_slopeFuzzy.save('o_slopeFuzzy')

In [12]:
#12. VI dostęp światła słonecznego
aspect = arcpy.sa.Aspect("nmt")
aspect.save("aspect")            # in decimal degrees !!!

aspect_sam_flat_issue = arcpy.sa.ExtractByMask(aspect, 'gmina_sam')
aspect_sam_flat_issue.save('aspect_sam_flat_issue')

aspect_sam = arcpy.sa.Con(arcpy.sa.Raster("aspect_sam_flat_issue") < -0.1, 180, "aspect_sam_flat_issue") # flat terrain is -1
aspect_sam.save("aspect_sam")

acpectFuzzy_1 = arcpy.sa.FuzzyMembership('aspect_sam', arcpy.sa.FuzzyLinear( 90, 112.5))
acpectFuzzy_1.save('acpectFuzzy_1')

acpectFuzzy_2 = arcpy.sa.FuzzyMembership('aspect_sam', arcpy.sa.FuzzyLinear( 270, 247.5))
acpectFuzzy_2.save('acpectFuzzy_2')

o_aspectFuzzy = arcpy.ia.Merge(['acpectFuzzy_1', 'acpectFuzzy_2'], "MIN")
o_aspectFuzzy.save('o_aspectFuzzy')

In [34]:
#13. VII węzły komunikacyjne
wezly_distance_raster = wezly_distance_raster_path

# add max distance field to gminaBuffer
arcpy.management.AddField("gminaBuffer", "kryt_6_max_dist", "INTEGER")
arcpy.management.CalculateField("gminaBuffer", "kryt_6_max_dist", f"10000", "PYTHON3")   # max distance from qgis processing

# rasterize gminaBuffer polygon
arcpy.conversion.PolygonToRaster("gminaBuffer", "kryt_6_max_dist", "gminaBufferRaster")

# extract by mask
wezly = arcpy.sa.ExtractByMask(wezly_distance_raster, "gminaBuffer")
wezly.save("wezly")

# merge rasters
wezlyDistance = arcpy.ia.Merge(["gminaBufferRaster", "wezly"], "MIN")
wezlyDistance.save("wezlyDistance")

wezlyDistance_sam = arcpy.sa.ExtractByMask(wezlyDistance, "gmina_sam")
wezlyDistance_sam.save("wezlyDistance_sam")

o_wezlyFuzzy = arcpy.sa.FuzzyMembership("wezlyDistance_sam", arcpy.sa.FuzzyLinear(4500, 300))
o_wezlyFuzzy.save("o_wezlyFuzzy")

In [9]:
#14. Łączenie kryteriów (równe wagi + ostre)
out_Fuzzy_equal = arcpy.ia.Raster("o_wodyFuzzy") + arcpy.ia.Raster("o_budynkiFuzzy") + arcpy.ia.Raster("o_lasyFuzzy") + arcpy.ia.Raster("o_drogiFuzzy") + arcpy.ia.Raster("o_slopeFuzzy") + arcpy.ia.Raster("o_aspectFuzzy") + arcpy.ia.Raster("o_wezlyFuzzy")
out_Fuzzy_equal.save("out_Fuzzy_equal")

out_Sharp = arcpy.ia.Raster("o_wodySharp") * arcpy.ia.Raster("o_budynkiSharp") * arcpy.ia.Raster("o_lasySharp")
out_Sharp.save("out_Sharp")

out_equal = arcpy.ia.Raster("out_Fuzzy_equal") * arcpy.ia.Raster("out_Sharp")
out_equal.save("out_equal")

In [ ]:
#15. różne wagi
# AHP                    wo   bu   la   dr   na   wy   wę
AHP_Matrix = np.array([ [1.0, 1.0, 1/3, 1.0, 1/3, 1/5, 3.0],  # woda
                        [1.0, 1.0, 1.0, 1.0, 1/2, 1/5, 3.0],  # budynki
                        [3.0, 1.0, 1.0, 2.0, 1.0, 1/3, 5.0],  # las
                        [1.0, 1.0, 1/2, 1.0, 1.0, 1/5, 1.0],  # drogi
                        [3.0, 2.0, 1.0, 1.0, 1.0, 1/3, 2.0],  # nachylenie
                        [5.0, 5.0, 3.0, 5.0, 3.0, 1.0, 5.0],  # wystawa
                        [1/3, 1/3, 1/5, 1.0, 1/2, 1/5, 1.0]], # węzły
                    dtype=np.float64)

column_weight_sums = np.sum(AHP_Matrix, axis=0)
print ("suma wag", column_weight_sums)
normalized_matrix = AHP_Matrix / column_weight_sums

control_sum = np.sum(normalized_matrix, axis=0)
row_weight_sums = np.sum(AHP_Matrix, axis=1)
summed_weights = np.sum(row_weight_sums)
mean_weights = row_weight_sums / summed_weights
print("wagi", mean_weights)
control_sum_2 = np.sum(mean_weights)
print("kontrola",control_sum_2)

out_Fuzzy_weighted = arcpy.sa.Raster("o_wodyFuzzy") * mean_weights[0] + arcpy.sa.Raster("o_budynkiFuzzy") * mean_weights[1] + arcpy.sa.Raster("o_lasyFuzzy") * mean_weights[2] + arcpy.sa.Raster("o_drogiFuzzy") * mean_weights[3] + arcpy.sa.Raster("o_slopeFuzzy") * mean_weights[4] + arcpy.sa.Raster("o_aspectFuzzy") * mean_weights[5] + arcpy.sa.Raster("o_wezlyFuzzy") * mean_weights[6]
out_Fuzzy_weighted.save("out_Fuzzy_weighted")

out_weighted = arcpy.sa.Raster("out_Fuzzy_weighted") * arcpy.sa.Raster("out_Sharp")
out_weighted.save("out_weighted")

In [19]:
#16. reklasyfikacja
output_equalReclassified = arcpy.sa.Reclassify(
    in_raster="out_equal",
    reclass_field="VALUE",
    remap="0 3.7 0;3.7 2077 1",
)
output_equalReclassified.save("output_equalReclassified")

output_weightedReclassified = arcpy.sa.Reclassify(
    in_raster="out_weighted",
    reclass_field="VALUE",
    # remap="0 0.6 0;0.6 1 1",
    # remap="0 0.56 0;0.56 1 1",
    remap="0 0.49 0;0.49 1 1",
)
output_weightedReclassified.save("output_weightedReclassified")

In [12]:
#17. działki
codeblock = """
def getSaturation(area, sum):
    if sum is None:
        return 0
    out = (sum * 25) / area
    if out < 1:
        return out
    else:
        return 1"""
for mode in ["equal", "weighted"]:
    dzialki_shp = dzialki_path
    arcpy.CopyFeatures_management(dzialki_shp, f"dzialki_{mode}_attr_issue")

    table_dzialki = arcpy.sa.ZonalStatisticsAsTable(f"dzialki_{mode}_attr_issue", "OBJECTID", f"output_{mode}Reclassified",
                                    f"table_dzialki_{mode}", ignore_nodata="DATA")

    # join fields
    arcpy.management.JoinField(f"dzialki_{mode}_attr_issue", "OBJECTID", f"table_dzialki_{mode}", "OBJECTID_1")

    # save
    arcpy.CopyFeatures_management(f"dzialki_{mode}_attr_issue", f"dzialki_{mode}")

    # saturation %
    arcpy.management.AddField(f"dzialki_{mode}", "saturation", "DOUBLE")
    expression = "getSaturation(!SHAPE.area!, !SUM!)"
    arcpy.management.CalculateField(f"dzialki_{mode}", "saturation", expression, "PYTHON3", codeblock)

    dzialkiProc = arcpy.SelectLayerByAttribute_management(f"dzialki_{mode}", "NEW_SELECTION", '"saturation" >= 0.6')
    arcpy.CopyFeatures_management(dzialkiProc, f"dzialkiProc_{mode}")

    arcpy.management.Dissolve(
        in_features=f"dzialkiProc_{mode}",
        out_feature_class=f"dzialkiDissolveMulti_{mode}",
        dissolve_field=None,
        statistics_fields=None,
        multi_part="MULTI_PART",
        unsplit_lines="DISSOLVE_LINES",
        concatenation_separator=""
    )

    arcpy.management.MultipartToSinglepart(f"dzialkiDissolveMulti_{mode}", f"dzialkiDissolve_{mode}")

    arcpy.management.MinimumBoundingGeometry(
        in_features=f"dzialkiDissolve_{mode}",
        out_feature_class=f"dzialkiMbgGeom_{mode}",
        geometry_type="CONVEX_HULL",
        group_option="NONE",
        group_field=None,
        mbg_fields_option="MBG_FIELDS"
    )

    arcpy.management.JoinField(
        in_data=f"dzialkiDissolve_{mode}",
        in_field="OBJECTID",
        join_table=f"dzialkiMbgGeom_{mode}",
        join_field="OBJECTID",
        fields="MBG_Width;MBG_Length",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )

    arcpy.CopyFeatures_management(f"dzialkiDissolve_{mode}", f"dzialkiMBG_{mode}")

    # 15 000 zamiast 20 000
    dzialkiWidthArea = arcpy.SelectLayerByAttribute_management(f"dzialkiMBG_{mode}", "NEW_SELECTION", '("MBG_Width" >= 50 AND "MBG_Length" >= 50) AND "SHAPE_Area" >= 15000')
    arcpy.CopyFeatures_management(dzialkiWidthArea, f"dzialkiWidthArea_{mode}")


In [ ]:
#18. ####### PTKM_A ########
# bufory
SKJZ_LBuffer = arcpy.analysis.Buffer("SKJZ_L", "SKJZ_LBuffer", "15 Meters", "FULL", "ROUND", "ALL")
SKTR_LBuffer = arcpy.analysis.Buffer("SKTR_L", "SKTR_LBuffer", "15 Meters", "FULL", "ROUND", "ALL")

# wartości i rasteryzacja
arcpy.management.AddField("SKTR_LBuffer", "ptkm_val", "INTEGER")
arcpy.management.CalculateField("SKTR_LBuffer", "ptkm_val", f"200", "PYTHON3")
arcpy.conversion.PolygonToRaster("SKTR_LBuffer", "ptkm_val", "SKTR_LRaster")

arcpy.management.AddField("SKJZ_LBuffer", "ptkm_val", "INTEGER")
arcpy.management.CalculateField("SKJZ_LBuffer", "ptkm_val", f"100", "PYTHON3")
arcpy.conversion.PolygonToRaster("SKJZ_LBuffer", "ptkm_val", "SKJZ_LRaster")

arcpy.management.AddField("gminaBuffer", "ptkm_val", "INTEGER")
arcpy.management.CalculateField("gminaBuffer", "ptkm_val", f"0", "PYTHON3")
arcpy.conversion.PolygonToRaster("gminaBuffer", "ptkm_val", "gminaRasterPTKM")

# merge
rasters_list = ["SKTR_LRaster", "SKJZ_LRaster", "gminaRasterPTKM"]
ptkm = arcpy.ia.Merge(rasters_list, "MAX")
ptkm.save("ptkm_raster")

# stats
table_ptkm = arcpy.sa.ZonalStatisticsAsTable("PTKM_A", "OBJECTID", "ptkm_raster", "table_ptkm", ignore_nodata="DATA")
arcpy.management.JoinField("PTKM_A", "OBJECTID", "table_ptkm", "OBJECTID_1")
# arcpy.CopyFeatures_management("PTKM_A", "PTKM_A_rodzaj")

# rodzaj
expression = "getRodzaj(!MAJORITY!)"
codeblock = """
def getRodzaj(maj):
    if maj is None:
        return 0
    elif maj == 200:
        return "torowisko"
    elif maj == 100:
        return "droga kołowa"
    else:
        return "inny teren komunikacyjny"
"""
arcpy.management.AddField("PTKM_A", "rodzaj", "TEXT")
arcpy.management.CalculateField("PTKM_A", "rodzaj", expression, "PYTHON3", codeblock)

In [30]:
#19. mapa kosztów względnych, noData = 0 
coverage_list = ["PTWP_A", "PTZB_A", "PTLZ_A", "PTRK_A", "PTUT_A", "PTTR_A", "PTKM_A", "PTGN_A", "PTPL_A", "PTSO_A", "PTWZ_A", "PTNZ_A"]
coverage_list_1 = coverage_list[:7]; coverage_list_2 = coverage_list[7:]
coverage_dict = {
    # PTWP_A
    "woda morska" : 0,
    "woda płynąca" : 200,
    "woda stojąca" : 0,
    # PTZB_A
    "jednorodzinna" : 100,
    "wielorodzinna" : 200,
    "pozostała zabudowa" : 50,
    "handlowo-usługowa" : 200,
    "przemysłowo-składowa" : 200,
    # PTLZ_A
    "las" : 100,
    "zagajnik" : 50,
    "zadrzewienie" : 50,
    # PTRK_A
    "kosodrzewina" : 15,
    "krzewy" : 15,
    # PTUT_A
    "sad" : 100,
    "plantacja" : 90,
    "szkółka roślin" : 20,
    "ogródki działkowe" : 0,
    # PTTR_A
    "roślinność trawiasta" : 20,
    "uprawa na gruntach ornych" : 1,
    # PTKM_A
    "torowisko" : 200,
    "droga kołowa" : 100,
    "inny teren komunikacyjny" : 0,

    # klasy nierozróżniane poprzez atrybut rodzaj
    "PTGN_A" : 1,
    "PTPL_A" : 50,
    "PTS0_A" : 0,
    "PTWZ_A" : 0,
    "PTNZ_A" : 150
}

arcpy.management.CreateFeatureclass(arcpy.env.workspace, "pokrycieVector", 'POLYGON')
arcpy.AddField_management("pokrycieVector", "value", "INTEGER")
with arcpy.da.InsertCursor("pokrycieVector", ["value", "SHAPE@"]) as insert_cursor:
    for cov in coverage_list_1:
        with arcpy.da.SearchCursor(cov, ["rodzaj", "SHAPE@"]) as cursor:
            for row in cursor:
                insert_cursor.insertRow([coverage_dict[row[0]], row[1]])
    
    for cov in coverage_list_2:
        with arcpy.da.SearchCursor(cov, ["SHAPE@"]) as cursor:
            for row in cursor:
                insert_cursor.insertRow([coverage_dict[cov], row[0]])
    
arcpy.conversion.FeatureToRaster("pokrycieVector", "value", "pokrycieRasterNotNull")
pokrycieRaster = arcpy.sa.SetNull("pokrycieRasterNotNull", "pokrycieRasterNotNull", "VALUE = 0")
pokrycieRaster.save("pokrycieRaster")

In [13]:
#20. cost
for mode in ["equal", "weighted"]:
    outCostDist = arcpy.sa.CostDistance(f"dzialkiWidthArea_{mode}", "pokrycieRaster", out_backlink_raster=f"pokrycieBacklink_{mode}")
    outCostDist.save(f"pokrycieCostDist_{mode}")

    costpathras = arcpy.sa.CostPath(
        in_destination_data="SULN_L",
        in_cost_distance_raster=f"pokrycieCostDist_{mode}",
        in_cost_backlink_raster=f"pokrycieBacklink_{mode}",
        path_type="BEST_SINGLE",
        destination_field="OBJECTID",
        force_flow_direction_convention="INPUT_RANGE"
    )
    costpathras.save(f"pokrycieCostPath_{mode}")

    arcpy.conversion.RasterToPolygon(
        in_raster=f"pokrycieCostPath_{mode}",
        out_polygon_features=f"pokrycieCostPathPolygon_{mode}",
        simplify="NO_SIMPLIFY",
        raster_field="Value",
        create_multipart_features="SINGLE_OUTER_PART",
        max_vertices_per_feature=None
    )
    
    the_dzialka_merged = arcpy.management.SelectLayerByLocation(f"dzialkiWidthArea_{mode}", "INTERSECT", f"pokrycieCostPathPolygon_{mode}")
    arcpy.management.CopyFeatures(the_dzialka_merged, f"dzialki_final_merged_{mode}")

In [ ]:
# produkt ostateczny
for mode in ["equal", "weighted"]:
    dzialki_final = arcpy.management.SelectLayerByLocation(f"dzialki_{mode}_attr_issue", "WITHIN", f"dzialki_final_merged_{mode}")
    arcpy.management.CopyFeatures(dzialki_final, f"dzialki_final_{mode}")

    with arcpy.da.SearchCursor(f"dzialki_final_{mode}", ["ID_DZIALKI"]) as cursor:
        print(f"_____działki_final_{mode}_____________________")
        for row in cursor:
            print(f"{row[0]}\n")
        print("______________________________________________")